In [79]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local') \
        .appName('Quiz2') \
        .config('spark.ui.port', '4050') \
        .getOrCreate()

sc = spark.sparkContext

데이터
- points1.csv: 10개의 점ID와 2차원 (xy좌표) 점의 집합
- points2.csv: 100,000개의 2차원 (xy좌표) 점의 집합
  
문제
- points1의 각 점 X에 대해 다른 points1의 점들 중 X를 가장 가까운 점으로 하는 points2의 점의 개수를 구하시오.
- 즉, 점 X의 ReverseClosestPoint(X)를 {u∈points2|∀v∈points1,d(u,X)≤d(u,v)}라는 집합으로 정의하였을 때 각 점 X에 대해 ReverseClosestPoint(X)의 크기를 출력하는 것이 문제임.
  
출력 
- 점ID, |ReverseClosestPoint(X)|
  
HINT
- points1은 broadcasting하고 ReverseClosestPoint(X)의 크기는 accumulator로 카운트 하시오.

In [84]:
point1 = sc.textFile('../../data/input2/points1.csv')
point1 = point1.map(lambda x: eval(x)).collect()
point1

[(0, 0.3332310446739237, 0.8816789623129827),
 (1, 0.7180093769486705, 0.9637912346703899),
 (2, 0.8116248725383151, 0.13696859152848095),
 (3, 0.32558100044331795, 0.503730643753619),
 (4, 0.8771663414457561, 0.7076376276477696),
 (5, 0.19388260220851417, 0.5493842742220786),
 (6, 0.27167090531975013, 0.43894203635672935),
 (7, 0.07368066232777415, 0.47765482567815376),
 (8, 0.9228510563432439, 0.05714995929075273),
 (9, 0.9140180890494094, 0.13546962101450588)]

In [85]:
bc_point_set = sc.broadcast(point1)

In [86]:
point2 = sc.textFile('../../data/input2/points2.csv')
point2.takeSample(False, 5)

['0.3605768939144025,0.41427340085592523',
 '0.49981193499908516,0.044445721194083965',
 '0.04282837853009491,0.8789469946706773',
 '0.26220726052348475,0.8162745170777618',
 '0.0223264683905956,0.7485061647576708']

In [87]:
def dist(x, y):
    return ((x[0] - y[0])**2 + (x[1] - y[1])**2)**0.5

def find_min_dist(point1, point2):
    point2 = list(map(float, point2.split(',')))
    minimum = 987654321
    minimum_idx = -1
    for idx, x, y in point1:
        d = dist((x, y), point2)
        if minimum > d:
            minimum = d
            minimum_idx = idx
    return minimum_idx

In [89]:
point2.map(lambda x: (find_min_dist(bc_point_set.value, x), x)).take(5)

[(1, '0.8071618868229837,0.9549318616231292'),
 (4, '0.8483020262221963,0.7199779417791973'),
 (6, '0.391731441893985,0.2280996037290972'),
 (2, '0.7217786652575899,0.3599210126660518'),
 (7, '0.006756387685526333,0.41676697341323576')]

In [90]:
point2.map(lambda x: (find_min_dist(bc_point_set.value, x), 1)).reduceByKey(lambda x, y: x + y).collect()

[(1, 8319),
 (4, 15936),
 (6, 14729),
 (2, 15088),
 (7, 8222),
 (3, 10818),
 (8, 1632),
 (5, 6201),
 (9, 4319),
 (0, 14736)]

In [91]:
sc.stop()